In [2]:
from datetime import datetime, timedelta
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, when, to_json, struct, lit, udf, collect_list, round as round_, max as max_, min as min_, sum as sum_, first, last
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, IntegerType, MapType
from pyspark.sql import functions as F
import boto3
import sys
from pandas import Timestamp
from pandas import DataFrame
import time
import io
import calendar
from operator import itemgetter
import freqtrade.vendor.qtpylib.indicators as qtpylib
import numpy as np  # noqa
import pandas as pd  # noqa
import pandas_ta as pta
import talib.abstract as ta
from dateutil.relativedelta import relativedelta
from freqtrade.strategy import IStrategy
from technical.indicators import *
from pandas import Timestamp
import requests
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import s3fs
import dotenv
from dotenv import load_dotenv
import os
import sys
import os
import findspark
findspark.init()

# Add the directory containing spark_config.py to the Python path
module_path = '/root/Cicada-binance/cores/aggTrades/historical/transformation/src/VolumeProfileCluster'
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the spark_config module
import spark_config
load_dotenv()

ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
REGION_NAME = os.getenv("AWS_REGION_NAME")
BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")


In [3]:
import sys
import os

# Set the SPARK_HOME and PATH environment variables
os.environ['SPARK_HOME'] = '/opt/spark'
os.environ['PATH'] += os.pathsep + os.path.join(os.environ['SPARK_HOME'], 'bin')

# Add the directory containing spark_config.py to the Python path
module_path = '/root/Cicada-binance/cores/aggTrades/historical/transformation/src/VolumeProfileCluster'
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the spark_config module
import spark_config

# Get the Spark session
spark_conf = spark_config.get_spark_session()

# Use boto3 and s3fs for additional S3 interaction
s3_resource = boto3.resource(
    's3',
    region_name=os.getenv("AWS_REGION_NAME"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

s3 = s3fs.S3FileSystem(anon=False, key=os.getenv("AWS_ACCESS_KEY_ID"), secret=os.getenv("AWS_SECRET_ACCESS_KEY"))

# Read data from S3
s3_path = 'xtrus31/cicada-data/DataTransformationAPI/binance-futures/BTCUSDT/aggTrade/15m/aggTrades_processed_2024-01_2024-05.parquet'
df = spark_conf.read.parquet(f"s3a://{s3_path}")


24/06/26 23:43:27 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/06/26 23:43:28 WARN BasicProfileConfigLoader: Your profile name includes a 'profile ' prefix. This is considered part of the profile name in the Java SDK, so you will need to include this prefix in your profile name when you reference this profile from your Java code.


In [4]:
df.show()

+-------------------+-------+-------+-------+-------+------------------+-----------------+-----------------+---------+
|       time_rounded|   open|  close|   high|    low|               qty|sum_bid_total_qty|sum_ask_total_qty|footprint|
+-------------------+-------+-------+-------+-------+------------------+-----------------+-----------------+---------+
|2024-01-01 01:00:00|42313.9|42532.5|42535.0|42289.6|3616.4240000000905|             NULL|             NULL|       {}|
|2024-01-01 01:15:00|42532.4|42458.5|42603.2|42449.1|2322.0279999999702|             NULL|             NULL|       {}|
|2024-01-01 01:30:00|42458.4|42474.5|42485.7|42386.2|1684.2169999999803|             NULL|             NULL|       {}|
|2024-01-01 01:45:00|42474.5|42503.5|42527.2|42449.1| 835.2439999999989|             NULL|             NULL|       {}|
|2024-01-01 02:00:00|42503.5|42497.6|42510.4|42462.0| 850.2740000000003|             NULL|             NULL|       {}|
|2024-01-01 02:15:00|42497.6|42524.8|42554.9|424

In [4]:
from config import config
from storage import s3Module
from VolumeProfileCluster import DataProcessor, spark_config
from VolumeProfileCluster.slack_package import (
    SlackChannel,
    get_slack_decorators,
)

In [6]:
# Initialize Spark session
spark = spark_config

slack_decorators = get_slack_decorators()
data_processor = DataProcessor(ACCESS_KEY, SECRET_KEY, BUCKET_NAME, REGION_NAME)
s3_module = s3Module(ACCESS_KEY, SECRET_KEY, BUCKET_NAME, REGION_NAME)

# Define the schema for bid and ask data
json_schema_bid = StructType(
    [
        StructField("price_level", DoubleType(), True),
        StructField("bid_qty", DoubleType(), True),
        StructField("bid_trades", IntegerType(), True),
        StructField("bid_trades_aggr", IntegerType(), True),
    ]
)

json_schema_ask = StructType(
    [
        StructField("price_level", DoubleType(), True),
        StructField("ask_qty", DoubleType(), True),
        StructField("ask_trades", IntegerType(), True),
        StructField("ask_trades_aggr", IntegerType(), True),
    ]
)

In [8]:
chart_type = "candlestick"
aggregate_trades = True

start_year = "2024"
start_month = "04"
end_year = "2024"
end_month = "05"
timeframe = "15m"
df = data_processor.calc_df(df, True, timeframe)

In [9]:
df.show()

+-------------------+-------+-------+-------+-------+------------------+--------------------+--------------------+
|       time_rounded|   open|  close|   high|    low|               qty|            foot_bid|            foot_ask|
+-------------------+-------+-------+-------+-------+------------------+--------------------+--------------------+
|2020-01-01 01:00:00|7189.43|7176.26|7190.52|7172.94|1037.3369999999989|[{"price_level":7...|[{"price_level":7...|
|2020-01-01 01:15:00|7176.22|7172.36|7179.41|7170.69| 707.8329999999991|[{"price_level":7...|[{"price_level":7...|
|2020-01-01 01:30:00|7172.79|7174.83|7179.45|7170.61| 325.2459999999999|[{"price_level":7...|[{"price_level":7...|
|2020-01-01 01:45:00|7174.51|7171.55|7179.36|7170.15| 378.6329999999997|[{"price_level":7...|[{"price_level":7...|
|2020-01-01 02:00:00|7171.43| 7186.6|7188.77| 7171.1| 555.3889999999992|[{"price_level":7...|[{"price_level":7...|
|2020-01-01 02:15:00| 7186.6| 7205.9| 7210.0|7184.16|1332.0769999999925|[{"price

In [10]:
df_bid_agg_sorted, df_bid_total_qty = data_processor.process_foot_data(
    df, "foot_bid", json_schema_bid, "bid"
)
df_bid_agg_sorted.show()
df_bid_total_qty.show()

+-------------------+-----------+------------------+----------------+---------------------+
|       time_rounded|price_level|     total_bid_qty|total_bid_trades|total_bid_trades_aggr|
+-------------------+-----------+------------------+----------------+---------------------+
|2020-01-01 01:00:00|     7173.0|17.825999999999997|              16|                   16|
|2020-01-01 01:00:00|     7174.0|15.869000000000002|              30|                   30|
|2020-01-01 01:00:00|     7175.0| 29.78100000000001|              39|                   39|
|2020-01-01 01:00:00|     7176.0| 51.82800000000002|              64|                   64|
|2020-01-01 01:00:00|     7177.0| 52.13700000000002|              60|                   60|
|2020-01-01 01:00:00|     7178.0|105.05199999999999|             100|                  100|
|2020-01-01 01:00:00|     7179.0| 71.12200000000001|              67|                   67|
|2020-01-01 01:00:00|     7180.0| 46.92099999999999|              54|           

+-------------------+------------------+
|       time_rounded| sum_bid_total_qty|
+-------------------+------------------+
|2020-01-01 01:00:00|           653.586|
|2020-01-01 01:15:00|           433.223|
|2020-01-01 01:30:00|162.71000000000004|
|2020-01-01 01:45:00|203.33200000000002|
|2020-01-01 02:00:00|           218.562|
|2020-01-01 02:15:00|           474.085|
|2020-01-01 02:30:00| 648.2780000000001|
|2020-01-01 02:45:00|           183.235|
|2020-01-01 03:00:00|           303.185|
|2020-01-01 03:15:00|           372.194|
|2020-01-01 03:30:00|           356.481|
|2020-01-01 03:45:00|422.35999999999996|
|2020-01-01 04:00:00|473.55799999999994|
|2020-01-01 04:15:00|187.30799999999996|
|2020-01-01 04:30:00|420.58700000000005|
|2020-01-01 04:45:00|           186.789|
|2020-01-01 05:00:00|           148.988|
|2020-01-01 05:15:00|191.71900000000002|
|2020-01-01 05:30:00|           132.815|
|2020-01-01 05:45:00|116.27999999999999|
+-------------------+------------------+
only showing top

# MAIN.PY

In [ ]:
import datetime
import json
import logging
import os
import tempfile
import time
import traceback

import boto3
import pytz
import schedule
from pyspark.sql import functions as F
from pyspark.sql.functions import col

from config import config
from storage import s3Module
from VolumeProfileCluster import DataProcessor, spark_config
from VolumeProfileCluster.slack_package import (
    SlackChannel,
    get_slack_decorators,
)

# Load environment variables
ACCESS_KEY = config.ACCESS_KEY
SECRET_KEY = config.SECRET_KEY
REGION_NAME = config.REGION_NAME
BUCKET_NAME = config.BUCKET_NAME
BEARER_TOKEN = config.BEARER_TOKEN
SLACK_WEBHOOK_URL = config.SLACK_WEBHOOK_URL
PROCESSED_FILES_LOG = config.PROCESSED_FILES_LOG

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("application.log"), logging.StreamHandler()],
)

# Initialize Spark session
spark = spark_config

slack_decorators = get_slack_decorators()
slack_channel = SlackChannel(SLACK_WEBHOOK_URL)
data_processor = DataProcessor(ACCESS_KEY, SECRET_KEY, BUCKET_NAME, REGION_NAME)
s3_module = s3Module(ACCESS_KEY, SECRET_KEY, BUCKET_NAME, REGION_NAME)


def transform_footprint(row):
    logging.info("Transforming footprint")
    try:
        bid_data = row["aggregated_foot_bid"]
        ask_data = row["aggregated_foot_ask"]
        aggregated_data = {}

        if bid_data:
            for bid_item in bid_data:
                for price, data in bid_item.items():
                    if price not in aggregated_data:
                        aggregated_data[price] = {
                            "bid_qty": 0,
                            "bid_trades": 0,
                            "ask_qty": 0,
                            "ask_trades": 0,
                        }
                    aggregated_data[price]["bid_qty"] += data["bid_qty"]
                    aggregated_data[price]["bid_trades"] += data["bid_trades"]

        if ask_data:
            for ask_item in ask_data:
                for price, data in ask_item.items():
                    if price not in aggregated_data:
                        aggregated_data[price] = {
                            "bid_qty": 0,
                            "bid_trades": 0,
                            "ask_qty": 0,
                            "ask_trades": 0,
                        }
                    aggregated_data[price]["ask_qty"] += data["ask_qty"]
                    aggregated_data[price]["ask_trades"] += data["ask_trades"]

        logging.info("Footprint transformation successful")
        return aggregated_data

    except Exception as e:
        logging.error(f"Error in transform_footprint: {e}")
        logging.error("Traceback details:", exc_info=True)
        return {}


@slack_decorators.notify_with_link(
    header="A new Parquet file is available 🗂️",
    message="The pipeline has updated the daily agg Trade file",
    color="#6a0dad",
)
def data_pipeline_exec(
    data_processor,
    df_trades,
    start_date,
    end_date,
    symbol,
    exchange,
    chart_type,
    timeframe,
    aggregate_trades=1,
):
    try:
        logging.info("Starting data pipeline execution")

        logging.info("Process footprint")
        processed_df = data_processor.calc_footprint(
            df_trades, True, chart_type, symbol, timeframe
        )

        logging.info("Process OHLCV")
        df_ohlcv = data_processor.calc_ohlcv(df_trades, timeframe)

        logging.info("Calculate df_bid_agg, df_ask_agg")
        df_bid_agg, df_bid_qty, df_ask_agg, df_ask_qty = (
            data_processor.aggregate_bid_ask_data(processed_df)
        )

        logging.info("Bid footprint transformation")
        df_footprint_bids = data_processor.transform_and_aggregate_footprint(
            df_bid_agg, "bid"
        )

        logging.info("Ask footprint transformation")
        df_footprint_asks = data_processor.transform_and_aggregate_footprint(
            df_ask_agg, "ask"
        )

        logging.info("Joining final dataframe")
        df_joined_final = data_processor.join_bid_ask_footprints(
            df_ohlcv, df_bid_qty, df_ask_qty, df_footprint_bids, df_footprint_asks
        )

        logging.info("Transforming footprint data in final dataframe")
        df_joined_final = df_joined_final.withColumn(
            "footprint",
            F.udf(lambda row: transform_footprint(row))(
                F.struct([df_joined_final[x] for x in df_joined_final.columns])
            ),
        )

        logging.info("Saving data to S3 and local filesystem")
        data_processor.save_dataframe_to_s3_fs(
            df_joined_final, symbol, exchange, timeframe, start_date, end_date
        )

        logging.info("Data pipeline execution completed successfully")
        return df_joined_final

    except Exception as e:
        logging.error(f"An exception occurred during data pipeline execution: {str(e)}")
        logging.error("Traceback details:", exc_info=True)
        raise


def run_data_pipeline(
    data_processor,
    df_trades,
    start_date,
    end_date,
    symbol,
    exchange,
    chart_type,
    timeframe,
):
    try:
        df_final = data_pipeline_exec(
            data_processor,
            df_trades,
            start_date,
            end_date,
            symbol,
            exchange,
            chart_type,
            timeframe,
        )
        logging.info("Data pipeline execution completed.")
    except Exception as e:
        logging.error(f"An exception occurred during data pipeline execution: {str(e)}")
        logging.error("Traceback details:", exc_info=True)


def process_year_data(
    data_processor,
    spark,
    s3_paths_by_year,
    year,
    symbol,
    exchange,
    chart_type,
    timeframes,
    start_month,
    end_month,
):
    try:
        logging.info(f"Processing data for year: {year}")

        df_year = None
        for month, s3_path in s3_paths_by_year[year]:
            try:
                logging.info(f"Reading data from {s3_path}")

                df_month = spark.read.parquet(s3_path)

                new_column_names = [
                    "agg_trade_id",
                    "price",
                    "quantity",
                    "first_trade_id",
                    "last_trade_id",
                    "time",
                    "is_buyer_maker",
                ]
                for old_name, new_name in zip(df_month.columns, new_column_names):
                    df_month = df_month.withColumnRenamed(old_name, new_name)

                header_row = df_month.filter(
                    col(new_column_names[0]) == "agg_trade_id"
                ).first()
                if header_row:
                    df_month = df_month.filter(
                        col(new_column_names[0]) != "agg_trade_id"
                    )

                if df_year is None:
                    df_year = df_month
                else:
                    df_year = df_year.union(df_month)

            except Exception as e:
                logging.error(f"Failed to read data from {s3_path}: {e}")

        if df_year is None:
            logging.info(f"No data available for year {year}. Skipping.")
            return

        for timeframe in timeframes:
            logging.info(f"Processing timeframe: {timeframe}")
            try:
                run_data_pipeline(
                    data_processor,
                    df_year,
                    f"{year}-{start_month:02d}",
                    f"{year}-{end_month:02d}",
                    symbol,
                    exchange,
                    chart_type,
                    timeframe,
                )
            except Exception as e:
                logging.error(
                    f"An exception occurred while processing data for {year}, {timeframe}: {e}"
                )
                logging.error("Traceback details:", exc_info=True)

    except Exception as e:
        logging.error(
            f"An exception occurred while processing data for year {year}: {e}"
        )
        logging.error("Traceback details:", exc_info=True)


def daily_update():
    start_time = time.time()

    chart_type = "candlestick"
    aggregate_trades = True

    start_year = "2024"
    start_month = "04"
    end_year = "2024"
    end_month = "05"

    paris_tz = pytz.timezone("Europe/Paris")
    current_time = datetime.datetime.now(tz=paris_tz)
    logging.info(f"Current Paris time: {current_time}")

    cryptos = s3_module.get_cryptos(
        BUCKET_NAME, "cicada-data/HistoricalTradeAggregator/binance_futures/"
    )
    timeframes = ["15mn"]

    all_s3_paths = []
    total_files = 0
    total_size = 0

    slack_channel.send_message(
        "🚀 Daily Update Initiated",
        f"_\"You think I'm a parasite, don't you? But I just want to make money.\"_ - Jared Vennett\n\n"
        f"*Starting the daily transformation with* `{len(cryptos)}` *cryptocurrencies across* `{len(timeframes)}` *timeframes.*\n"
        f"🕰️ *Start Time*: {current_time}",
    )

    for crypto in cryptos:
        crypto_start_time = time.time()
        crypto_files = 0
        crypto_size = 0

        s3_paths = s3_module.generate_s3_paths(
            crypto.replace("/", ""), start_year, end_year, start_month, end_month
        )
        all_s3_paths.extend(s3_paths)

        s3_paths_by_year = {}
        for year, month, s3_path in all_s3_paths:
            if year not in s3_paths_by_year:
                s3_paths_by_year[year] = []
            s3_paths_by_year[year].append((month, s3_path))

        for year in s3_paths_by_year:
            year_start_month = int(start_month) if year == int(start_year) else 1
            year_end_month = int(end_month) if year == int(end_year) else 12
            process_year_data(
                data_processor,
                spark,
                s3_paths_by_year,
                year,
                crypto,
                "binance-futures",
                chart_type,
                timeframes,
                year_start_month,
                year_end_month,
            )

        crypto_files += len(s3_paths)

        total_files += crypto_files
        total_size += 0

        crypto_duration = time.time() - crypto_start_time
        slack_channel.send_message(
            f"✅ Crypto Transformation Complete: {crypto}",
            f'_"This business kills the part of life that is essential, the part that has nothing to do with business."_ - Ben Rickert\n\n'
            f"Successfully processed *{crypto_files}* files for *{crypto}* in *{crypto_duration:.2f} seconds*. The data now occupies a total of *{crypto_size / (1024 ** 3):.2f} GB*.\n"
            f"📈 *Files Processed*: {crypto_files}\n"
            f"🕰️ *Processing Time*: {crypto_duration:.2f} seconds\n"
            f"💾 *Total Data Size*: {crypto_size / (1024 ** 3):.2f} GB",
        )

    total_duration = time.time() - start_time
    slack_channel.send_message(
        "🎬 Daily Transformation Complete",
        f'_"In the end they knew."_ - Narrator\n\n'
        f"All cryptocurrencies have been processed. Here are the final stats for today's update:\n"
        f"📈 *Total Files Processed*: {total_files}\n"
        f"🕰️ *Total Processing Time*: {total_duration / 60:.2f} minutes\n"
        f"💾 *Total Data Size*: {total_size / (1024 ** 3):.2f} GB\n"
        f"Outstanding work, everyone! 🌟",
    )


schedule.every().day.at("10:00").do(daily_update)

schedule.run_all()

while True:
    schedule.run_pending()
    time.sleep(1)
